In [122]:
import pandas as pd
import csv
import clvehicles
import os
from itertools import combinations
from datetime import datetime
import math
import numpy as np

Enter in the make and model below. I created a list of some of the major cities in the US. The loop will create a unique csv for each set of city, make, and model.

In [2]:
#TODO make CLI capable of accepting cities with 2 words (i.e. New York, los, angeles)
cities = ['Chicago', 'Phoenix', 'Madison', 'Austin']
make = 'Acura'
model = 'MDX'

frames = []
df = pd.DataFrame()

for i in cities:
    os.system('python clvehicles.py ' + i + ' ' + make + ' ' + model)
    filename = i + model + '.csv'
    temp_df = pd.read_csv(filename)
    df.append(temp_df, ignore_index = True)
    frames.append(temp_df)
    
df = pd.concat(frames, axis=0)

In [23]:
#this command stops the df from truncating the url. It makes debugging by viewing the original posting much easier
pd.set_option('display.max_colwidth', -1)

In [193]:
#df[df['location'] == 'austin']
df[df['location'] == 'chicago'].head(100)


,name,price,year,condition,milage,title status,transmission,drive,cylinders,fuel,color,location,timestamp,url
0,2009 ACURA MDX,$12500,2009,excellent,NaN,clean,automatic,NaN,6 cylinders,gas,white,chicago,2018-03-21 22:32,https://chicago.craigslist.org/sox/cto/d/2009-acura-mdx/6539134249.html
1,ACURA MDX,$12500,2009,excellent,NaN,clean,automatic,4wd,6 cylinders,gas,white,chicago,2018-03-21 22:31,https://chicago.craigslist.org/sox/cto/d/acura-mdx/6539136288.html
2,2008 Acura MDX AWD W/3rd Row,$8400,2008,NaN,NaN,clean,automatic,4wd,NaN,gas,NaN,chicago,2018-03-21 19:15,https://chicago.craigslist.org/sox/cto/d/2008-acura-mdx-awd-3rd-row/6538984711.html
3,2004 Acura MDX,$4900,2004,excellent,NaN,clean,automatic,NaN,NaN,gas,NaN,chicago,2018-03-21 18:20,https://chicago.craigslist.org/chc/cto/d/2004-acura-mdx/6520990166.html
4,2011 Acura MDX 6-Spd AT w/Tech Package **100% approval is our goal**,$16988,2011,NaN,116972.0,clean,other,NaN,NaN,gas,NaN,chicago,2018-03-21 16:54,https://chicago.craigslist.org/chc/ctd/d/2011-acura-mdx-6-spd-at-tech/6538842206.html
5,2003 Acura MDX,$2999,2003,NaN,NaN,clean,automatic,NaN,NaN,gas,NaN,chicago,2018-03-21 13:52,https://chicago.craigslist.org/chc/cto/d/2003-acura-mdx/6538603414.html
6,2007 ACURA MDX AWD,$7200,2007,NaN,NaN,clean,automatic,NaN,NaN,gas,NaN,chicago,2018-03-21 12:48,https://chicago.craigslist.org/chc/cto/d/2007-acura-mdx-awd/6538513069.html
7,2006 Acura MDX clean Acura!! text me 3125487540 - Blue,$7395,2006,NaN,0.0,clean,automatic,NaN,6 cylinders,gas,blue,chicago,2018-03-21 11:40,https://chicago.craigslist.org/ctd/d/2006-acura-mdx-clean-acura/6538422476.html
8,2011 ACURA MDX,$15800,2011,like new,NaN,clean,automatic,fwd,6 cylinders,gas,grey,chicago,2018-03-21 11:36,https://chicago.craigslist.org/chc/ctd/d/2011-acura-mdx/6538417610.html
9,2008 ACURA MDX AWD SUNROOF HEATED SEATS LEATHER ***WARRANTY***,$8995,2008,excellent,107499.0,clean,automatic,4wd,6 cylinders,gas,white,chicago,2018-03-21 10:38,https://chicago.craigslist.org/nwi/ctd/d/2008-acura-mdx-awd-sunroof/6538334826.html


In [16]:
#df.iloc[250]['location']
df.iloc[250]

name                                 2016 Acura MDX w/ Towing Pkg
price                                                      $30500
year                                                         2016
condition                                                like new
milage                                                      32000
title status                                                clean
transmission                                            automatic
drive                                                         NaN
cylinders                                                     NaN
fuel                                                          gas
color                                                         NaN
location                                                   austin
timestamp                                        2018-03-08 14:54
url             https://austin.craigslist.org/cto/d/2016-acura...
Name: 14, dtype: object

The next step is to determine how to eliminate duplicate rows. 

In [41]:
before_drop = len(df)
df.drop_duplicates()
after_drop = len(df)
display(str(before_drop - after_drop) +  ' duplicates were dropped')

'0 duplicates were dropped'

Because the URL is different on identical listings, it is not a good metric to determine duplicates. Furthermore, because we have aggregated all of the data by model and make, it is unreasonable to eliminate rows that have similar entries.
For example: https://phoenix.craigslist.org/wvl/cto/d/2007-acura-mdx-awd/6538180578.html and https://phoenix.craigslist.org/wvl/ctd/d/2007-acura-mdx/6537533572.html have the same model, make, year, price, location, and title status but are different listings. The only two feasible ways to eliminate duplicate rows is by comparing the images on two different pages, or comparing the times they were posted. Most duplicates are posted within a couple minutes of each other and have similar titles, so that will be the criteria for determining rows that should be deleted.

In [192]:
#TODO
#Fix these functions. Not all duplicates were posted at the same time. Sometimes listings have the same name
#and are different cars. Other times they have different names and are the same car. Currently working on this.

def rowsAreSimilar(input_df, row1, row2):
    if input_df.name.iloc[row1] == input_df.name.iloc[row2]:
        return True
    if input_df.year.iloc[row1] == input_df.year.iloc[row2]:
        return True
    #Need more parameters
    return False


def eliminateRowsByTime(input_df, city):
    length = len(input_df)
    lst = np.arange(0,length,1)
    elims = []
    for combo in combinations(lst, 2):  #selects each combination (saves runtime rather than testing rows 0 vs 1 AND 1 vs 0.)
        #print(combo[1])
        time1  = (df[df['location'] == city].timestamp[combo[0]])
        time2  = (df[df['location'] == city].timestamp[combo[1]])
        dtime1 = datetime.strptime(time1, '%Y-%m-%d %H:%M')
        dtime2 = datetime.strptime(time2, '%Y-%m-%d %H:%M')
        diff = (dtime1 - dtime2)
        
        '''This block is executed if there are two rows with almost identical timestamps. 
        Now we compare the column values to see if they are the same car'''
        if (diff.total_seconds()/60) < 1.5: #if the difference of timestamps of the postings is <1.5 minutes
            if rowsAreSimilar(input_df, combo[0], combo[1]): #if the rows are similar
                elims.append(combo[1]) #has to be a duplicate posting
    print('elims are', elims)

c = df[df['location'] == 'chicago']
eliminateRowsByTime(c, 'chicago')

('elims are', [1, 19, 73, 99])


In [108]:
len(df[df['location'] == 'austin'])

27

In [183]:
df.name.iloc[1]

'ACURA MDX'